In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
%cd /content/drive/My Drive/Colab Notebooks/Github/predicting_components_source/PredictingComponent-multilabel
!ls

/content/drive/My Drive/Colab Notebooks/Github/predicting_components_source/PredictingComponent-multilabel
Classifier  experiment_predict_component_result.csv  NCE       test2.txt
Dataset     __init__.py				     Other_DL


**Set experiment parameters**

In [0]:
project = 'HIVE'

trainingSize = 60
validationSize = 20
testSize = 20

**Load csv of the whole dataset**

In [8]:
import numpy as np
import pandas as pd

# /content/drive/My Drive/Colab Notebooks/Github/predicting_components_source/PredictingComponent-multilabel/Dataset/Data/HBASE.csv

data_path = 'Dataset/Data/' + project + '.csv'
data_path
data = pd.read_csv(data_path)
data.shape

(9358, 69)

In [0]:
# data.head(3)

**Remove unused columns and concat summary and description**

In [0]:
data['description'] = data['title'] + data['description']
data = data.drop('title',1) 
data = data.drop('issuekey',1) 
data = data.drop('component',1) 

In [11]:
data.head(3)

,description,API,Accumulo Storage Handler,Authentication,Authorization,Beeline,Build Infrastructure,CBO,CLI,Clients,Compression,Configuration,Contrib,Database/Schema,Diagnosability,Documentation,Encryption,File Formats,HBase Handler,HBase Metastore,HCatalog,Hive,HiveServer2,Import/Export,Indexing,JDBC,Locking,Logging,Logical Optimizer,Macros,Metastore,ODBC,OLAP,ORC,PTF-Windowing,Parser,Physical Optimizer,Query Planning,Query Processor,SQL,SQLStandardAuthorization,Security,Serializers/Deserializers,Server Infrastructure,Shims,Spark,Statistics,StorageHandler,Test,Testing Infrastructure,Tests,Tez,Thrift API,Transactions,Types,UDF,Vectorization,Views,Web UI,WebHCat,Website,Windows,hpl/sql,llap,spark-branch,storage-api
0,Hive should be able to create tables over bina...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,support for reading binary data from flat file...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Enable findbugs and hive tests from hadoop bui...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
data.description[0]

'Hive should be able to create tables over binary flat filesCurrently Hive only supports binary data embedded inside SequenceFiles. This is a request to enable this over flat files (for example a flat file containings sets of serialized binary records).  It is understood that this file cannot be split and that the user had to ensure that there is enough file level parallelism. maybe related to https://issues.apache.org/jira/browse/HADOOP-3787.  Hive also does not play nice with generic hadoop serialization mechanism (1986) - however that requires much more comprehensive change to SerDe interfaces.'

**Split training/test set**

In [13]:
if trainingSize + validationSize + testSize == 100:
  numData = len(data)
  numTrain = int((trainingSize * numData) / 100)
  numValidation = int((validationSize * numData) / 100)
  numTest = int((testSize * numData) / 100)

  print("#Total size: %s" % numData)
  print("#Training : %s, #Validation : %s, #Testing : %s" % (numTrain, numValidation, numTest))
  print("Total: %s" % (numTrain + numValidation + numTest))

  firstStop = numTrain
  secondStop = numTrain + numValidation
else:
  print("input not valid")


#Total size: 9358
#Training : 5614, #Validation : 1871, #Testing : 1871
Total: 9356


In [14]:
divided_set = np.zeros([numData, 3]).astype(int)
divided_set[0:firstStop, 0] = 1
divided_set[firstStop:secondStop, 1] = 1
divided_set[secondStop:numData, 2] = 1
experimentalSet = pd.DataFrame(divided_set, columns = ['train', 'validate', 'test'])
print(experimentalSet)

      train  validate  test
0         1         0     0
1         1         0     0
2         1         0     0
3         1         0     0
4         1         0     0
...     ...       ...   ...
9353      0         0     1
9354      0         0     1
9355      0         0     1
9356      0         0     1
9357      0         0     1

[9358 rows x 3 columns]


In [15]:
trainSetIndex = experimentalSet['train'].value_counts()[1]
validSetIndex = experimentalSet['validate'].value_counts()[1]
testSetIndex = experimentalSet['test'].value_counts()[1]
print("Training set index: " + str(trainSetIndex))
print("Validation set index: " + str(validSetIndex))
print("Test set index: " + str(testSetIndex))

Training set index: 5614
Validation set index: 1871
Test set index: 1873


In [16]:
trainSetData = data.iloc[:trainSetIndex, :]
validSetData = data.iloc[trainSetIndex:trainSetIndex + validSetIndex, :]
testSetData = data.iloc[trainSetIndex + validSetIndex:, :]

print("Training set:")
trainSetData.head(3)


Training set:


,description,API,Accumulo Storage Handler,Authentication,Authorization,Beeline,Build Infrastructure,CBO,CLI,Clients,Compression,Configuration,Contrib,Database/Schema,Diagnosability,Documentation,Encryption,File Formats,HBase Handler,HBase Metastore,HCatalog,Hive,HiveServer2,Import/Export,Indexing,JDBC,Locking,Logging,Logical Optimizer,Macros,Metastore,ODBC,OLAP,ORC,PTF-Windowing,Parser,Physical Optimizer,Query Planning,Query Processor,SQL,SQLStandardAuthorization,Security,Serializers/Deserializers,Server Infrastructure,Shims,Spark,Statistics,StorageHandler,Test,Testing Infrastructure,Tests,Tez,Thrift API,Transactions,Types,UDF,Vectorization,Views,Web UI,WebHCat,Website,Windows,hpl/sql,llap,spark-branch,storage-api
0,Hive should be able to create tables over bina...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,support for reading binary data from flat file...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Enable findbugs and hive tests from hadoop bui...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
print("Validation set:")
validSetData.head(3)

Validation set:


,description,API,Accumulo Storage Handler,Authentication,Authorization,Beeline,Build Infrastructure,CBO,CLI,Clients,Compression,Configuration,Contrib,Database/Schema,Diagnosability,Documentation,Encryption,File Formats,HBase Handler,HBase Metastore,HCatalog,Hive,HiveServer2,Import/Export,Indexing,JDBC,Locking,Logging,Logical Optimizer,Macros,Metastore,ODBC,OLAP,ORC,PTF-Windowing,Parser,Physical Optimizer,Query Planning,Query Processor,SQL,SQLStandardAuthorization,Security,Serializers/Deserializers,Server Infrastructure,Shims,Spark,Statistics,StorageHandler,Test,Testing Infrastructure,Tests,Tez,Thrift API,Transactions,Types,UDF,Vectorization,Views,Web UI,WebHCat,Website,Windows,hpl/sql,llap,spark-branch,storage-api
5614,Reading an ACID table with vectorization on re...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
5615,Update statement supports multi-table synatxDe...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5616,Delete statement supports multi-table syntaxDe...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
print("testset:")
testSetData.head(3)

testset:


,description,API,Accumulo Storage Handler,Authentication,Authorization,Beeline,Build Infrastructure,CBO,CLI,Clients,Compression,Configuration,Contrib,Database/Schema,Diagnosability,Documentation,Encryption,File Formats,HBase Handler,HBase Metastore,HCatalog,Hive,HiveServer2,Import/Export,Indexing,JDBC,Locking,Logging,Logical Optimizer,Macros,Metastore,ODBC,OLAP,ORC,PTF-Windowing,Parser,Physical Optimizer,Query Planning,Query Processor,SQL,SQLStandardAuthorization,Security,Serializers/Deserializers,Server Infrastructure,Shims,Spark,Statistics,StorageHandler,Test,Testing Infrastructure,Tests,Tez,Thrift API,Transactions,Types,UDF,Vectorization,Views,Web UI,WebHCat,Website,Windows,hpl/sql,llap,spark-branch,storage-api
7485,insert values clause should support functionsi...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7486,Hive Metastore does not tolerate leading space...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7487,Avro SerDe InstanceCache returns incorrect sch...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Wording embedding layer using pretrained model from Google's word2vec**

In [19]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
trainText = trainSetData.description
validText = validSetData.description
testText = testSetData.description

In [21]:
NUM_WORDS=300000
tokenizer = Tokenizer(num_words=NUM_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'', lower=True)
tokenizer.fit_on_texts(trainText)
trainSequences = tokenizer.texts_to_sequences(trainText)
validSequences = tokenizer.texts_to_sequences(validText)
testSequences = tokenizer.texts_to_sequences(testText)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 24851 unique tokens.


In [22]:
x_train = pad_sequences(trainSequences)
y_train = trainSetData.iloc[:,1:].values

x_valid = pad_sequences(validSequences, maxlen = x_train.shape[1])
y_valid = validSetData.iloc[:,1:].values

x_test = pad_sequences(testSequences,maxlen = x_train.shape[1])
y_test = testSetData.iloc[:,1:].values

print('Shape of X train and X validation tensor:', x_train.shape, x_valid.shape)
print('Shape of label train and validation tensor:', y_train.shape, y_valid.shape)


print('Shape of X test:', x_test.shape)
print('Shape of label test tensor:', y_test.shape)

Shape of X train and X validation tensor: (5614, 2804) (1871, 2804)
Shape of label train and validation tensor: (5614, 65) (1871, 65)
Shape of X test: (1873, 2804)
Shape of label test tensor: (1873, 65)


**word embedding**

In [0]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

In [24]:
word_vectors = KeyedVectors.load_word2vec_format('Other_DL/pretrainedw2v/GoogleNews-vectors-negative300.bin.gz', 
                                                 binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [25]:
EMBEDDING_DIM=300
vocabulary_size = min(len(word_index)+1,NUM_WORDS) # minimum fo word index and num words
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

count_word_not_found = 0
for word, i in word_index.items():
  if i>=NUM_WORDS:
    continue
  try:
    embedding_vector = word_vectors[word]
    embedding_matrix[i] = embedding_vector
  except KeyError:
    embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)
    count_word_not_found = count_word_not_found + 1

print("word not found in pretrained w2v: " + str(count_word_not_found))
# there are many words do not exist in google w2v. we random number for those words.

del(word_vectors)


word not found in pretrained w2v: 16958


**Construct CNN model**

In [26]:
from keras.layers import Embedding

embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)
                            
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau


sequence_length = x_train.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
output = Dense(units=len(y_train[0]), activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

# this creates a model that includes
model = Model(inputs, output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2804)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 2804, 300)    7455600     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 2804, 300, 1) 0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 2802, 1, 100) 90100       reshape_1[0][0]                  
____________________________________________________________________________________________

**Train the model**

In [27]:
path_modelcheckpoint = 'Other_DL/model_checkpoint/'
adam = Adam(lr=1e-3)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
model_checkpoint = ModelCheckpoint(path_modelcheckpoint+project+'_mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')


model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])

training_history = model.fit(x_train, y_train, batch_size=100, epochs=100, verbose=1, validation_data=(x_valid, y_valid), 
                             callbacks=[early_stopping, model_checkpoint, reduce_lr_loss])  # starts training


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 5614 samples, validate on 1871 samples
Epoch 1/100
5614/5614 [==============================] - 1114s 198ms/step - loss: 4.7001 - acc: 0.3192 - val_loss: 5.3598 - val_acc: 0.2090

Epoch 00001: val_loss improved from inf to 5.35981, saving model to Other_DL/model_checkpoint/HIVE_mdl_wts.hdf5
Epoch 2/100
5614/5614 [==============================] - 1113s 198ms/step - loss: 4.0071 - acc: 0.3962 - val_loss: 5.2279 - val_acc: 0.3709

Epoch 00002: val_loss improved from 5.35981 to 5.22786, saving model to Other_DL/model_checkpoint/HIVE_mdl_wts.hdf5
Epoch 3/100
5614/5614 [==============================] - 1112s 198ms/step - loss: 3.9690 - acc: 0.4289 - val_loss: 6.2485 - val_acc: 0.4148

Epoch 00003: val_loss did not improve from 5.22786
Epoch 4/100
5614/5614 [==============================] - 1113s 198ms/step - loss: 4.5584 - acc: 0.4195 - val_loss: 9.6261 - val_acc: 0.4169

Epoch 00004: val_loss did not improve from 5.22786
Epoch 5/100
5614/5614 [==============================] - 1

**Save model**

In [28]:
import pandas as pd
path_history = 'Other_DL/training_history/'
path_model_json = 'Other_DL/model_structure/'
path_model_weight = 'Other_DL/model_weight/'

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(training_history.history) 

# or save to csv: 
hist_csv_file = path_history+project+'_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

# save the best model
# save model structure
model_json = model.to_json()
with open(path_model_json+project+"_model.json", "w") as json_file:
    json_file.write(model_json)
# save model weight
model.save_weights(path_model_weight+project+"_wieght.h5")
print("Saved model to disk")


Saved model to disk


In [29]:
# # later...

from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy as np
import pandas as pd

path_history = 'Other_DL/training_history/'
path_model_json = 'Other_DL/model_structure/'
path_model_weight = 'Other_DL/model_weight/'

# # load json and create model
json_file = open(path_model_json+project+"_model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# # load weights into new model
model.load_weights(path_model_weight+project+"_wieght.h5")
print("Loaded model from disk")

# # evaluate loaded model on test data
# loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# score = loaded_model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk


**Make a recommendation on the testset**

In [0]:
y_pred = model.predict(x_test)

***Save result***

In [0]:
path_log = 'Other_DL/output/log/'

np.savetxt(path_log + project + "_actual.csv", y_test, delimiter=",")
np.savetxt(path_log + project + "_estimate.csv", y_pred, delimiter=",")

***Evaluation***

In [0]:
def recall(actual, estimate, startK, stopK, stepK):
    recall_k = []
    for k in range(startK, stopK + 1, stepK):
        if k == 0:
            k = 1
        m = len(actual)
        sum_recall = 0.0
        for j in range(len(actual)):
            y_true = np.argwhere(actual[j])
            y_pred = estimate[j].argsort()[-k:]
            # print y_true
            # print y_pred
            intersect = (y_true == y_pred).sum()
            # print intersect
            # print (len(y_true))
            # print intersect / float(len(y_true))
            sum_recall = sum_recall + (intersect / float(len(y_true)))
            # print sum_recall
        recall_k.append(sum_recall / float(m))
        # print m
        print('Recall@' + str(k) + ': {:.4f}'.format(sum_recall / float(m)))
    return recall_k

In [33]:
startK = 1
stepK = 1
stopK = 30

print("Project:" + project)

path_output = 'Other_DL/output/'

recall_k = recall(y_test, y_pred, startK, stopK, stepK)

np.savetxt(path_output + project + "_recall_" + str(startK) + "_" + str(stopK)+ "_v.csv", recall_k, delimiter=",", fmt='%1.4f')
with open(path_output + "performance" + "_recall_" + str(startK) + "_" + str(stopK)+ ".csv", 'a') as myoutput:
  myoutput.write(project + "," + ",".join(map(str, recall_k)) + '\n')



Project:HIVE
Recall@1: 0.1431
Recall@2: 0.2113
Recall@3: 0.2587
Recall@4: 0.2833
Recall@5: 0.2968
Recall@6: 0.3017
Recall@7: 0.3068
Recall@8: 0.3155
Recall@9: 0.3314
Recall@10: 0.3441
Recall@11: 0.3552
Recall@12: 0.3642
Recall@13: 0.3713
Recall@14: 0.3771
Recall@15: 0.3979
Recall@16: 0.4026
Recall@17: 0.4103
Recall@18: 0.4164
Recall@19: 0.4228
Recall@20: 0.4315
Recall@21: 0.4381
Recall@22: 0.4489
Recall@23: 0.4591
Recall@24: 0.4677
Recall@25: 0.4734
Recall@26: 0.5075
Recall@27: 0.5310
Recall@28: 0.5379
Recall@29: 0.5475
Recall@30: 0.5597
